## 结合工具规划逻辑

包括 ReAct、ReWoo、Plan 等复杂的工具规划推理也可以通过 FlowAgent 实现。

## ReAct

### 查天气

In [1]:
from illufly.chat import ChatQwen, ReAct
from illufly.types import Messages
print(ReAct.__doc__)

Messages(["hi"]).messages[-1].content


    ReAct 提供了一种更易于人类理解、诊断和控制的决策和推理过程。
    它的典型流程可以用一个有趣的循环来描述：思考（Thought）→ 行动（Action）→ 观察（Observation），简称TAO循环。
    


'hi'

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(ChatQwen(name="qwen", tools=[get_city, get_weather, booking]))
flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=True)

flow.last_output

[AGENT] >>> Node 1: qwen
[INFO] 记住 10 轮对话
**思考**  
首先，我需要确定鸿蒙公司位于哪个城市。因此，我的第一步应该是使用 `get_city` 工具来获取鸿蒙公司的位置信息。

**行动**  
Step1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]
[USAGE] {"input_tokens": 456, "output_tokens": 67, "total_tokens": 523}
广州
[FINAL_TOOL_RESP] 广州
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州
[AGENT] >>> Node 1: qwen
[INFO] 记住 10 轮对话
**思考**  
既然已经知道鸿蒙公司位于广州，那么接下来的步骤应该是使用 `get_weather` 工具来查询广州的天气情况。

**行动**  
Step2: 查询广州的天气状况. #E2 = get_weather[{"city": "广州"}]
[USAGE] {"input_tokens": 534, "output_tokens": 60, "total_tokens": 594}
广州天气不错，一直是大晴天
[FINAL_TOOL_RESP] 广州天气不错，一直是大晴天
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] >>> Node 1: qwen
[INFO] 记住 10 轮对话
**思考**  
我已经获得了鸿蒙公司所在的城市（广州）以及该城市的天气情况（大晴天）。基于此信息，我认为没有必要采取进一步的行动，可以直接给出最终答案。

**最终答案**  
鸿蒙公司所在的广州天气非常好，一直是大晴天。
[USAGE] {"input_tokens": 612, "output_tokens": 58, "total_tokens": 670}
[AGENT] >>> Node 2: observer
[INFO] 执行完毕，所有节点运行 6 步


'鸿蒙公司所在的广州天气非常好，一直是大晴天。'

In [3]:
flow.agents[0].last_output

'**思考**  \n我已经得到了鸿蒙公司位于广州的信息，并且也获取了广州的天气情况，即“广州天气不错，一直是大晴天”。基于这些信息，我认为没有必要再进行进一步的查询或行动，可以直接给出最终答案了。\n\n**最终答案**  \n鸿蒙公司的所在地广州天气很好，一直保持晴朗。希望这能帮到您！如果有其他需要了解的信息，请随时告知。'

In [6]:
flow.agents[1].completed_work

['**思考**  \n首先，我需要知道鸿蒙公司位于哪个城市，才能进一步查询该地区的天气情况。因此，我的第一步应该是确定鸿蒙公司的具体位置。\n\n**行动**  \nStep1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]',
 '\n**观察**\n上面的行动结果为:\n广州\n',
 '**思考**  \n既然已经知道了鸿蒙公司位于广州，接下来我应该查询广州的天气情况。\n\n**行动**  \nStep1: 查询广州的天气情况. #E1 = get_weather[{"city": "广州"}]',
 '\n**观察**\n上面的行动结果为:\n广州天气不错，一直是大晴天\n',
 '**思考**  \n我已经获得了鸿蒙公司所在的地理位置信息，即广州，同时我也查询到了广州的天气情况，即“广州天气不错，一直是大晴天”。基于这些信息，我认为可以结束任务并向用户报告天气情况了。\n\n**最终答案**\n您在鸿蒙公司所在地——广州的天气情况非常好，一直保持晴朗。希望这样的好天气能为您带来愉快的一天！']

In [2]:
print(flow.agents[0].memory)

[{'role': 'system', 'content': '尽你所能完成任务。\n\n**你要解决的问题是: ** 我在鸿蒙公司，帮我查询一下天气情况\n\n\n**思考**  \n首先，我需要知道鸿蒙公司位于哪个城市，才能进一步查询该地区的天气情况。因此，我的第一步应该是确定鸿蒙公司的具体位置。\n\n**行动**  \nStep1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]\n\n**观察**\n上面的行动结果为:\n广州\n\n**思考**  \n既然已经知道了鸿蒙公司位于广州，接下来我应该查询广州的天气情况。\n\n**行动**  \nStep1: 查询广州的天气情况. #E1 = get_weather[{"city": "广州"}]\n\n**观察**\n上面的行动结果为:\n广州天气不错，一直是大晴天\n\n\n(现在不要急于解决问题，而是继续按照如下步骤一步一步输出你的推理过程。)\n\n**思考** \n对当前情况进行反思, 然后说明你现在的决策：当前就结束任务并输出**最终答案**，还是继续考虑下一步行动的应当如何执行。\n\n**最终答案**\n(在此输出你的最终答案。)\n如果输出最终答案，就结束任务，停止所有输出。\n\n**行动** \n你可以尝试直接解决问题，并输出解决的详细过程；\n或者从 ** get_city,get_weather,booking ** 中选择一个工具来解决问题。\n\n这些工具的详细描述如下:\n{"type": "function", "function": {"name": "get_city", "description": "由任意地名或地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}\n{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters":

In [6]:
print(flow.agents[1].consumer_dict['task'])

我在鸿蒙公司，帮我查询一下天气情况


### 订球场

In [2]:
flow("我在鸿蒙公司，如果不下雨就帮我订个球场")

[AGENT] STEP 1 >>> Node 1: ChatQwen.4428024992
**思考**  
首先，我需要知道鸿蒙公司所在的地点，以便确定具体城市，之后才能查询该城市的天气情况。因此，我的第一步行动是使用 get_city 工具来确定鸿蒙公司所在的城市。

**行动**  
Plan: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]
广州

**观察** 上面的行动结果为:
广州
[AGENT] STEP 2 >>> Node 1: ChatQwen.4428024992
**思考**  
既然已经得知鸿蒙公司位于广州，接下来我需要查询广州的天气情况，以判断是否需要预订球场。因此，我的下一步行动是使用 get_weather 工具来获取广州的天气信息。

**行动**  
Plan: 查询广州的天气状况. #E2 = get_weather[{"city": "广州"}]
广州
广州天气不错，一直是大晴天

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天
[AGENT] STEP 3 >>> Node 1: ChatQwen.4428024992
**思考**  
既然广州的天气是大晴天，没有下雨，那么接下来应该预订一个球场。因此，我的下一步行动是使用 booking 工具来预订一个球场。

**行动**  
Plan: 预订一个球场. #E3 = booking[{"request": "在广州预订一个球场"}]
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快
[AGENT] STEP 4 >>> Node 1: ChatQwen.4428024992
**思考**  
根据之前获得的信息，广州天气晴朗，适合户外活动，而且已经成功预订了一个球场。因此，现在可以结束任务了，告知用户球场已预订好，并祝其玩得愉快。

**最终答案**
我已经为你在广州预订好了一个球场，天气晴朗，非常适合户外运动，希望你能玩得开心！


'我在鸿蒙公司，如果不下雨就帮我订个球场'

### 查看提示语组装结果

In [3]:
print(flow.agents[0].memory[0]['content'])

尽你所能完成任务。

**你要解决的问题是: ** 我在鸿蒙公司，如果不下雨就帮我订个球场


**思考**  
首先，我需要知道鸿蒙公司所在的地点，以便了解该地区的天气情况。因此，我的第一个步骤应该是使用 `get_city` 工具来获取鸿蒙公司所在的城市。

**行动**  
Plan: 获取鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察** 上面的行动结果为:
广州

**思考**  
既然已经知道了鸿蒙公司位于广州，接下来我需要查看广州的天气情况，以确定是否需要预订球场。

**行动**  
Plan: 查看广州的天气状况. #E2 = get_weather[{"city": "广州"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天

**思考**  
既然广州的天气是大晴天，没有下雨的情况，那么接下来的步骤应该是预订球场。

**行动**  
Plan: 预订球场. #E3 = booking[{"request": "在广州订个球场"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快


(现在不要急于解决问题，而是继续按照如下步骤一步一步输出你的推理过程。)

**思考** 
对当前情况进行反思, 然后说明你现在的决策：当前就结束任务并输出**最终答案**，还是继续考虑下一步行动的应当如何执行。

**最终答案**
(在此输出你的最终答案。)
如果输出最终答案，就结束任务，停止所有输出。

**行动** 
你可以尝试直接解决问题，并输出解决的详细过程；
或者从 ** get_city,get_weather,booking ** 中选择一个工具来解决问题。

这些工具的详细描述如下:
{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}
{

### 查看思考过程

In [4]:
print(flow.provider_dict['completed_work'])

**思考**  
首先，我需要知道鸿蒙公司所在的地点，以便了解该地区的天气情况。因此，我的第一个步骤应该是使用 `get_city` 工具来获取鸿蒙公司所在的城市。

**行动**  
Plan: 获取鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察** 上面的行动结果为:
广州

**思考**  
既然已经知道了鸿蒙公司位于广州，接下来我需要查看广州的天气情况，以确定是否需要预订球场。

**行动**  
Plan: 查看广州的天气状况. #E2 = get_weather[{"city": "广州"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天

**思考**  
既然广州的天气是大晴天，没有下雨的情况，那么接下来的步骤应该是预订球场。

**行动**  
Plan: 预订球场. #E3 = booking[{"request": "在广州订个球场"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快

**思考**  
基于之前的行动和观察结果，我们知道广州的天气非常好，没有下雨，而且球场已经成功预订了。在这种情况下，不需要进一步采取行动，可以结束任务了。

**最终答案**  
球场已经成功预订，祝你在广州的活动顺利，玩得开心！


In [5]:
flow.final_answer

'球场已经成功预订，祝你在广州的活动顺利，玩得开心！'

## ReWOO

- 管道计算场景：上一个任务的输出，作为下一个任务的输入
- 参数组装场景：上一个任务的输出需要做一定转换，才能作为下一个任务的输入（执行哪个任务是确定的，但参数需要组装）
- 条件分支场景：上一个任务的输出作为一种判定条件，决定执行接下来哪个任务（执行哪个任务不确定，需要根据当前任务返回来判定）

### 查天气

In [2]:
flow.agents

(<ChatQwen qwen>, <Observer observer>, <Selector should_continue>)

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReWOO
def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReWOO(
    ChatQwen(name="planner", tools=[get_city, get_weather, booking]),
    ChatQwen(name="solver")
)
flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=True)

print(flow.agents[1].provider_dict['completed_work'])
flow.last_output

[AGENT] >>> Node 1: planner
[INFO] 记住 10 轮对话
Step1: 首先需要确定鸿蒙公司所在的地址或城市。#E1 = get_city[{"location": "鸿蒙公司"}]
Step2: 使用上一步获取到的城市信息查询天气情况。#E2 = get_weather[{"city": "#E1"}]
[USAGE] {"input_tokens": 511, "output_tokens": 62, "total_tokens": 573}
广州
[FINAL_TOOL_RESP] 广州
广州天气不错，一直是大晴天
[FINAL_TOOL_RESP] 广州天气不错，一直是大晴天
[AGENT] >>> Node 2: observer
[AGENT] >>> Node 3: solver
[INFO] 记住 10 轮对话
```markdown
**最终答案**
广州天气不错，一直是大晴天
```
[USAGE] {"input_tokens": 199, "output_tokens": 17, "total_tokens": 216}
[INFO] 执行完毕，所有节点运行 2 步
Step1: 首先需要确定鸿蒙公司所在的地址或城市。
#E1 = get_city[{"location": "鸿蒙公司"}]
#E1 执行结果: 广州

Step2: 使用上一步获取到的城市信息查询天气情况。
#E2 = get_weather[{"city": "#E1"}]
#E2 执行结果: 广州天气不错，一直是大晴天



'广州天气不错，一直是大晴天'

In [3]:
flow.last_output

### 查看规划

In [3]:
flow.handler_tool_call.steps

[{'id': '#E1',
  'description': '首先，我们需要确定鸿蒙公司的位置所在城市。',
  'name': 'get_city',
  'arguments': '{"location": "鸿蒙公司"}',
  'result': '广州'},
 {'id': '#E2',
  'description': '然后，我们根据得到的城市信息查询该城市的天气情况。',
  'name': 'get_weather',
  'arguments': '{"city": "#E1"}',
  'result': '广州天气不错，一直是大晴天'}]

### 查看已完成工作

In [4]:
print(flow.handler_tool_call.completed_work)

Plan: 首先，我们需要确定鸿蒙公司的位置所在城市。 
#E1 = get_city[{"location": "鸿蒙公司"}] 
get_city Value = 广州
Plan: 然后，我们根据得到的城市信息查询该城市的天气情况。 
#E2 = get_weather[{"city": "#E1"}] 
get_weather Value = 广州天气不错，一直是大晴天


### 查看 Planner 提示语组装结果

In [5]:
print(flow.agents[0].memory[0]['content'])

对于以下任务，制定可以逐步解决问题的计划。
对于每个计划，指明使用哪个外部工具以及工具输入来获取证据。
你可以将具体计划的执行结果存储在一个变量 #E 中，后续工具可以调用该变量。
(Plan, #E1, Plan, #E2, Plan, ...)

工具可以是以下之一：
{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}
{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}
{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}

**Example:**
任务: Thomas、Toby 和 Rebecca 在一周内总共工作了 157 小时。
Thomas 工作了 x 小时。
Toby 工作的时间比 Thomas 的两倍少 10 小时，而 Rebecca 工作的时间比 Toby 少 8 小时。
Rebecca 工作了多少小时？

Plan: 假设 Thomas 工作了 x 小时，将问题转化为代数表达式并使用 Wolfram Alpha 解决。#E1 =

### 查看 Solver 提示语组装结果

In [6]:
print(flow.solver.memory[0]['content'])

解决以下任务或问题。
为了解决这个问题，我们制定了逐步计划，并获得了每个计划的执行结果。
请谨慎使用这些执行结果，因为其中可能包含不相关的信息。

Plan: 首先，我们需要确定鸿蒙公司的位置所在城市。 
#E1 = get_city[{"location": "鸿蒙公司"}] 
get_city Value = 广州
Plan: 然后，我们根据得到的城市信息查询该城市的天气情况。 
#E2 = get_weather[{"city": "#E1"}] 
get_weather Value = 广州天气不错，一直是大晴天

现在根据上面提供的执行结果解决问题或任务。
直接回答问题，不要废话，不要评论。

任务: 我在鸿蒙公司，帮我查询一下天气情况

你的回答:



### 复杂情况：订球场

In [2]:
flow("如果不下雨，你帮我订一个球场？")

[AGENT] STEP 1 >>> Node 1: ChatQwen.4561190192
Plan: 首先，我们需要确定用户的位置信息以便查询当地的天气情况。#E1 = get_city({"location": "用户位置"})
Plan: 根据获得的城市信息查询天气预报。#E2 = get_weather({"city": "#E1"})
Plan: 如果天气预报显示不会下雨，那么我们就可以预订球场。#E3 = booking({"request": "如果 #E2 表明没有雨，则预订一个球场"})
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快
我已经帮你预订好了，祝你玩得愉快。

**最终答案**
我已经帮你预订好了，祝你玩得愉快。

'我已经帮你预订好了，祝你玩得愉快。'

In [10]:
flow.planner.memory

[{'role': 'system',
  'content': '对于以下任务，制定可以逐步解决问题的计划。\n对于每个计划，指明使用哪个外部工具以及工具输入来获取证据。\n你可以将具体计划的执行结果存储在一个变量 #E 中，后续工具可以调用该变量。\n(Plan, #E1, Plan, #E2, Plan, ...)\n\n工具可以是以下之一：\n{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}\n{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}\n{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}\n\n**Example:**\n任务: Thomas、Toby 和 Rebecca 在一周内总共工作了 157 小时。\nThomas 工作了 x 小时。\nToby 工作的时间比 Thomas 的两倍少 10 小时，而 Rebecca 工作的时间比 Toby 少 8 小时。\nRebecca 工作了多少小时？\n\nPlan: 假设 Tho

## Plan And Execute

### 提示语测试

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

qwen = ChatQwen(memory=PromptTemplate("FLOW/PlanAndExe/Planner"), tools=[get_city, get_weather, booking])

task = "我要回公司，看看我是否需要带伞，另外帮我订一个羽毛球场"
qwen(task, new_chat=True)

In [2]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

plan = """
Step1: 使用 get_city 函数通过公司地址确定所在城市 #E1 Value = 广州
Step2: 使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2
Step3: 根据 #E2 的天气情况判断是否需要带伞 #E3
Step4: 使用 booking 函数预订羽毛球场 #E4
"""

step2 = "使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2"

qwen = ChatQwen(tools=[get_city, get_weather, booking])
qwen([("user", PromptTemplate("FLOW/PlanAndExe/Worker", binding_map={"plan": plan, "step_n": 2}))])



[TOOLS_CALL_FINAL] [{"index": 0, "id": "call_ac6cf72f876747d99f3e7b", "type": "function", "function": {"name": "get_weather", "arguments": "{\"city\": \"广州\"}"}}]
广州天气不错，一直是大晴天
好的，已经查询到广州今天的天气情况是大晴天。


'好的，已经查询到广州今天的天气情况是大晴天。'

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

plan = """
Step1: 使用 get_city 函数通过公司地址确定所在城市 #E1 Value = 广州
Step2: 使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2
Step3: 根据 #E2 的天气情况判断是否需要带伞 #E3
Step4: 使用 booking 函数预订羽毛球场 #E4
"""

completed_work = """
Step1: 使用 get_city 函数通过公司地址确定所在城市 #E1
Step2: 使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2
"""

task = "我要回公司，看看我是否需要带伞，另外帮我订一个羽毛球场"

qwen = ChatQwen(memory=PromptTemplate("FLOW/PlanAndExe/RePlanner"), tools=[get_city, get_weather, booking])
qwen.bind_provider({"completed_work": completed_work, "plan": plan, "task": task})
qwen("请开始", new_chat=True)

Step3: 根据 #E2 的天气情况判断是否需要带伞 #E3
Step4: 使用 booking 函数预订羽毛球场 #E4


'Step3: 根据 #E2 的天气情况判断是否需要带伞 #E3\nStep4: 使用 booking 函数预订羽毛球场 #E4'

In [1]:
from illufly.types import BaseAgent, PromptTemplate

plan = """
Step1: 使用 get_city 函数通过公司地址确定所在城市 #E1 Value = 广州
Step2: 使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2
Step3: 根据 #E2 的天气情况判断是否需要带伞 #E3
Step4: 使用 booking 函数预订羽毛球场 #E4
"""

completed_work = """
Step1: 使用 get_city 函数通过公司地址确定所在城市 #E1
Step2: 使用 get_weather 函数查询 #E1 所得城市今天的天气情况 #E2
"""

task = "我要回公司，看看我是否需要带伞，另外帮我订一个羽毛球场"

print(
    PromptTemplate(
        "FLOW/PlanAndExe/Planner",
        binding_map = {
            "task": task
        }
    ).format()
)

结合可用的工具，为给定目标制定一个简单的逐步执行的计划。
这个计划应包括各个任务，如果正确执行，将得出正确答案。
不要添加任何多余的步骤。
最终步骤的结果应该是最终答案。
确保每个步骤都有所需的所有信息 - 不要跳过步骤。

你可以从  中选择一个或多个工具使用。这些工具的详细描述为：



你输出的计划必须将执行结果存储在一个变量 #E{n} 中，任务间的依赖关系可以在任务秒速中引用该变量来反应。
(Step1, #E1, Step2, #E2, Step3, ...)

其中 #E{n} 用于保存计划执行后的变量名，n 是子任务的序号，格式为:
Step{n}: (子任务描述) #E{n}
每个计划后面应只跟一个 #E{n}。

例子：
Step1: (详细的计划描述) #E1
Step2: (详细的计划描述) #E2
...



In [3]:
print(
    PromptTemplate(
        "OUTLINE",
        binding_map = {
            "task": task
        }
    ).format()
)

你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
3. 请按照需要输出一份写作提纲
4. 提纲的构成应当由多个阔系额要求构成
5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等
5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹
6. 你只能输出提纲，不要输出具体的扩写内容

**你的任务是:**
我要回公司，看看我是否需要带伞，另外帮我订一个羽毛球场


**输出例子1**
```
# 第一章 标题1
## 一、XXXX
<OUTLINE>
扩写摘要:
对扩写内容做摘要总结

扩写要求：
- xxx
- xxx
</OUTLINE>

## 二、XXXX
<OUTLINE>
</OUTLINE>

## （更多扩写提纲）
```


